In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from pickle import load
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from model.nns import *
from model.bls import BroadLearningSystem
from model.elm import ELMRegressor, GenELMRegressor
from model.random_layer import RandomLayer, MLPRandomLayer, RBFRandomLayer, GRBFRandomLayer
from model.pyESN import ESN
from evaluation import *# metric_calculation, pred_bls_elm, pred_esn, load_model_config
from sklearn.manifold import TSNE


%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=6

2024-02-19 18:08:25.018353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=6


In [2]:
%matplotlib inline
rcParams["figure.subplot.wspace"] = 0.05
rcParams["figure.subplot.hspace"] = 0.15
rcParams["figure.titlesize"] = 'xx-large'
rcParams["figure.titleweight"] = 'bold'
rcParams['axes.titlesize'] = 'xx-large'
rcParams['axes.titleweight'] = 'bold'
rcParams['axes.labelsize'] = 'xx-large'
rcParams['axes.labelweight'] = 'bold'
rcParams["ytick.labelsize"] = 'xx-large'
rcParams["xtick.labelsize"] = 'xx-large'
rcParams["axes.xmargin"] = 0.05

### Evaluation


In [ ]:
results = plot_comparison(
    "recurrent",
    figsize=(74, 18),
    fontsize=30,
    linewidth=5,
    anchor=(0, -0.08, 1, 1),
    legsize=25,
)
display(results)

train_samples (6800, 1500, 3), test_inputs (1201, 100, 3), y_true (1201, 1400, 3)
here 2 lorenz/0.05, BLS pred shape is (1201, 200, 3); y_true shape is (1201, 200, 3)
BLS on lorenz/0.05:, y_true shape (1201, 200, 3); y_pred shape (1201, 200, 3)
here 2 lorenz/0.05, ELM pred shape is (1201, 200, 3); y_true shape is (1201, 200, 3)
ELM on lorenz/0.05:, y_true shape (1201, 200, 3); y_pred shape (1201, 200, 3)
lorenz/0.05, SSFS, pred shape is (1201, 300, 3); y_true shape is (1201, 200, 3); inference_steps 200
here 2 lorenz/0.05, SSFS pred shape is (1201, 200, 3); y_true shape is (1201, 200, 3)
SSFS on lorenz/0.05:, y_true shape (1201, 200, 3); y_pred shape (1201, 200, 3)
lorenz/0.05, HF5, pred shape is (1201, 300, 3); y_true shape is (1201, 200, 3); inference_steps 200
here 2 lorenz/0.05, HF5 pred shape is (1201, 200, 3); y_true shape is (1201, 200, 3)
HF5 on lorenz/0.05:, y_true shape (1201, 200, 3); y_pred shape (1201, 200, 3)
lorenz/0.05, HF10, pred shape is (1201, 300, 3); y_true shape i

2024-02-19 18:08:28.530132: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: UNKNOWN ERROR (34)


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
no trained model available at saved_models/pre_trained/dwelling_worm/2/fold_1/False/1/discontinuous_scheduled_sampling_is.h5
no trained model available at saved_models/pre_trained/dwelling_worm/2/fold_1/False/1/horizon_forcing_5.h5
no trained model available at saved_models/pre_trained/dwelling_worm/2/fold_1/False/1/horizon_forcing_10.h5
no trained model available at saved_models/pre_trained/dwelling_worm/2/fold_1/False/1/horizon_forcing_15.h5
no trained model available at saved_models/pre_trained/dwelling_worm/2/fold_1/Fa

/tf/workspace/yong.zhuang001/HorizonForcing/model/bls.py:59: RuntimeWarning: overflow encountered in exp
  return (2 / (1 + np.exp(-2 * x))) - 1
